In [1]:
import tkinter as tk
import mysql.connector
from mysql.connector import Error

# Create a connection to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='11Mfld11',
            database='mystic_schema'
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

In [2]:
# Create a new chat room
def create_chat_room():
    room_name = room_name_entry.get()
    creator_id = creator_id_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Insert a new chat room
        insert_query = "INSERT INTO ChatRooms (ChatRoomName, CreatedByUserID) VALUES (%s, %s)"
        cursor.execute(insert_query, (room_name, creator_id))
        connection.commit()

        cursor.close()
        connection.close()

# Create the Tkinter GUI window
root = tk.Tk()
root.title("Chat Room Manager")

# Create labels and entry fields for chat room creation
room_name_label = tk.Label(root, text="Chat Room Name:")
room_name_label.pack()
room_name_entry = tk.Entry(root)
room_name_entry.pack()

creator_id_label = tk.Label(root, text="Created By (User ID):")
creator_id_label.pack()
creator_id_entry = tk.Entry(root)
creator_id_entry.pack()

create_button = tk.Button(root, text="Create Chat Room", command=create_chat_room)
create_button.pack()

root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\mysql\connector\connection_cext.py", line 633, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Cannot add or update a child row: a foreign key constraint fails (`mystic_schema`.`chatrooms`, CONSTRAINT `chatrooms_ibfk_1` FOREIGN KEY (`CreatedByUserID`) REFERENCES `characters` (`id`))

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\fabia\AppData\Local\Temp\ipykernel_6072\3209159742.py", line 12, in create_chat_room
    cursor.execute(insert_query, (room_name, creator_id))
  File "c:\Users\fabia\AppData\Local\Programs\Python\Python311\Lib\site-packages\mysql\connector\cursor_cext.

In [4]:
import tkinter as tk
import mysql.connector
from mysql.connector import Error

chat_room_listbox = None

# Create a connection to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='11Mfld11',
            database='mystic_schema'
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Fetch existing chat rooms from the database
def fetch_chat_rooms():
    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Fetch chat room names from the ChatRooms table
        select_query = "SELECT ChatRoomName FROM ChatRooms"
        cursor.execute(select_query)
        chat_rooms = [row[0] for row in cursor.fetchall()]

        cursor.close()
        connection.close()

        return chat_rooms
    return []

# Join an existing chat room
def join_chat_room():
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    player_name = player_name_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Get the ChatRoomID based on the selected room's name
        select_query = "SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s"
        cursor.execute(select_query, (selected_room,))
        room_id = cursor.fetchone()[0]

        # Get the UserID based on the player's name from the Characters table
        select_user_query = "SELECT ID FROM Characters WHERE Name = %s"
        cursor.execute(select_user_query, (player_name,))
        user_id = cursor.fetchone()

        if user_id:
            user_id = user_id[0]
            # Check if the player is already a member
            check_query = "SELECT COUNT(*) FROM ChatRoomMembers WHERE ChatRoomID = %s AND UserID = %s"
            cursor.execute(check_query, (room_id, user_id))
            count = cursor.fetchone()[0]

            if count == 0:
                # Insert the player as a member of the chat room
                insert_query = "INSERT INTO ChatRoomMembers (ChatRoomID, UserID) VALUES (%s, %s)"
                cursor.execute(insert_query, (room_id, user_id))
                connection.commit()
                print(f"Player {user_id} joined the chat room {selected_room}")
            else:
                print(f"Player {user_id} is already a member of the chat room {selected_room}")
        else:
            print(f"Player {player_name} not found in the Characters table.")

        cursor.close()
        connection.close()

# Send a message in the selected chat room
def send_message(message_entry, sender_name_entry):
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    message_text = message_entry.get()
    sender_name = sender_name_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Get the ChatRoomID based on the selected room's name
        select_query = "SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s"
        cursor.execute(select_query, (selected_room,))
        room_id = cursor.fetchone()[0]

        # Get the UserID based on the sender's name from the Characters table
        select_user_query = "SELECT ID FROM Characters WHERE Name = %s"
        cursor.execute(select_user_query, (sender_name,))
        user_id = cursor.fetchone()

        if user_id:
            user_id = user_id[0]
            # Insert the message into the ChatMessages table
            insert_query = "INSERT INTO ChatMessages (ChatRoomID, UserID, MessageContent) VALUES (%s, %s, %s)"
            cursor.execute(insert_query, (room_id, user_id, message_text))
            connection.commit()
        else:
            print(f"Sender {sender_name} not found in the Characters table.")

        cursor.close()
        connection.close()

# Create the Tkinter GUI window for chat interaction
def chat_interaction():
    global chat_room_listbox  # Define chat_room_listbox as global
    root = tk.Tk()
    root.title("Chat Interaction")

    # Create a listbox to display available chat rooms
    chat_room_listbox = tk.Listbox(root)
    chat_room_listbox.pack()

    # Fetch existing chat rooms from the database
    chat_rooms = fetch_chat_rooms()
    for room in chat_rooms:
        chat_room_listbox.insert(tk.END, room)

    # Create labels and entry fields for joining chat rooms
    player_name_label = tk.Label(root, text="Player Name:")
    player_name_label.pack()
    player_name_entry = tk.Entry(root)
    player_name_entry.pack()

    join_button = tk.Button(root, text="Join Chat Room", command=join_chat_room)
    join_button.pack()

    # Create labels and entry fields for sending messages
    message_label = tk.Label(root, text="Message:")
    message_label.pack()
    message_entry = tk.Entry(root)
    message_entry.pack()

    sender_name_label = tk.Label(root, text="Sender Name:")
    sender_name_label.pack()
    sender_name_entry = tk.Entry(root)
    sender_name_entry.pack()

    send_button = tk.Button(root, text="Send Message", command=send_message(message_entry, sender_name_entry))
    send_button.pack()

    root.mainloop()

# Create the main program to choose chat interaction
def main():
    chat_interaction()

if __name__ == "__main__":
    main()

ProgrammingError: 1054 (42S22): Unknown column 'Name' in 'where clause'

In [2]:
import tkinter as tk
import mysql.connector
from mysql.connector import Error

def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='11Mfld11',
            database='mystic_schema'
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Fetch existing chat rooms from the database
def fetch_chat_rooms():
    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Fetch chat room names from the ChatRooms table
        select_query = "SELECT ChatRoomName FROM ChatRooms"
        cursor.execute(select_query)
        chat_rooms = [row[0] for row in cursor.fetchall()]

        cursor.close()
        connection.close()

        return chat_rooms
    return []

# Join an existing chat room
def join_chat_room(chat_room_listbox,player_id_entry):
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    player_id = player_id_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Get the ChatRoomID based on the selected room's name
        select_query = "SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s"
        cursor.execute(select_query, (selected_room,))
        room_id = cursor.fetchone()[0]

        # Check if the player is already a member
        check_query = "SELECT COUNT(*) FROM ChatRoomMembers WHERE ChatRoomID = %s AND UserID = %s"
        cursor.execute(check_query, (room_id, player_id))
        count = cursor.fetchone()[0]

        if count == 0:
            # Insert the player as a member of the chat room
            insert_query = "INSERT INTO ChatRoomMembers (ChatRoomID, UserID) VALUES (%s, %s)"
            cursor.execute(insert_query, (room_id, player_id))
            connection.commit()
            print(f"Player {player_id} joined the chat room {selected_room}")
        else:
            print(f"Player {player_id} is already a member of the chat room {selected_room}")

        cursor.close()
        connection.close()

# Send a message in the selected chat room
def send_message(message_entry, sender_id_entry):
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    message_text = message_entry.get()
    sender_id = sender_id_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Get the ChatRoomID based on the selected room's name
        select_query = "SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s"
        cursor.execute(select_query, (selected_room,))
        room_id = cursor.fetchone()[0]

        # Insert the message into the ChatMessages table
        insert_query = "INSERT INTO ChatMessages (ChatRoomID, UserID, MessageContent) VALUES (%s, %s, %s)"
        cursor.execute(insert_query, (room_id, sender_id, message_text))
        connection.commit()

        cursor.close()
        connection.close()

# Create the Tkinter GUI window for chat interaction
def chat_interaction():
    root = tk.Tk()
    root.title("Chat Interaction")

    # Create a listbox to display available chat rooms
    chat_room_listbox = tk.Listbox(root)
    chat_room_listbox.pack()

    # Fetch existing chat rooms from the database
    chat_rooms = fetch_chat_rooms()
    for room in chat_rooms:
        chat_room_listbox.insert(tk.END, room)

    # Create labels and entry fields for joining chat rooms
    player_id_label = tk.Label(root, text="Player (User ID):")
    player_id_label.pack()
    player_id_entry = tk.Entry(root)
    player_id_entry.pack()

    join_button = tk.Button(root, text="Join Chat Room", command=join_chat_room)
    join_button.pack()

    # Create labels and entry fields for sending messages
    message_label = tk.Label(root, text="Message:")
    message_label.pack()
    message_entry = tk.Entry(root)
    message_entry.pack()

    sender_id_label = tk.Label(root, text="Sender (User ID):")
    sender_id_label.pack()
    sender_id_entry = tk.Entry(root)
    sender_id_entry.pack()

    send_button = tk.Button(root, text="Send Message", command=send_message(message_entry, sender_id_entry))
    send_button.pack()

    root.mainloop()

# Create the main program to choose chat interaction
def main():
    chat_interaction()

if __name__ == "__main__":
    main()


NameError: name 'chat_room_listbox' is not defined